# CellMage Ambient Mode Testing

This notebook demonstrates how to properly initialize and use CellMage's ambient mode functionality.
Created: 25 April 2025

## 1. First, let's load the extension and verify it's working

In [1]:
# Load the extension
%load_ext cellmage.integrations.ipython_magic

# Verify that it's working by showing the current status
%llm_config --status

✅ NotebookLLM Magics loaded. Use %llm_config and %%llm.
   For ambient mode, try %llm_config_persistent to process all cells as LLM prompts.
--- NotebookLLM Status ---
Session ID: b279d502-dd34-4ad3-989d-26ba6bb1a12c
None
Active Overrides: {'api_key': 'sk-L...mA', 'api_base': 'https://litellm.oracle.madpin.dev', 'model': 'gpt-4.1-nano'}
History Length: 0 messages
--------------------------


## 2. Debug NotebookLLMMagics Registration

Let's check if the NotebookLLMMagics class is properly registered with IPython

In [2]:
# Debug the registered magic commands
import sys
from cellmage.integrations.ipython_magic import NotebookLLMMagics

ip = get_ipython()
print("Available magic types:", list(ip.magics_manager.magics.keys()))

# Check all registered magics for NotebookLLMMagics instances
found = False
for magic_type, magics in ip.magics_manager.magics.items():
    for name, instance in magics.items():
        if isinstance(instance, NotebookLLMMagics):
            print(f"✅ Found NotebookLLMMagics instance registered as {magic_type} magic: {name}")
            found = True

if not found:
    print("❌ NotebookLLMMagics instance not found in registered magics")

Available magic types: ['line', 'cell']
❌ NotebookLLMMagics instance not found in registered magics


## 3. Fix the Registration Issue

If NotebookLLMMagics is not found, let's try re-registering it manually

In [3]:
# Force manual registration of NotebookLLMMagics
from IPython import get_ipython
from cellmage.integrations.ipython_magic import NotebookLLMMagics, load_ipython_extension

ip = get_ipython()

# First, unload if already registered
%reload_ext cellmage.integrations.ipython_magic

# Then manually call the load function
print("Manually calling load_ipython_extension...")
load_ipython_extension(ip)

# Check if registration was successful
found = False
for magic_type, magics in ip.magics_manager.magics.items():
    for name, instance in magics.items():
        if isinstance(instance, NotebookLLMMagics):
            print(f"✅ Found NotebookLLMMagics instance registered as {magic_type} magic: {name}")
            found = True
            
if not found:
    print("❌ Still couldn't register NotebookLLMMagics")

✅ NotebookLLM Magics loaded. Use %llm_config and %%llm.
   For ambient mode, try %llm_config_persistent to process all cells as LLM prompts.
Manually calling load_ipython_extension...
✅ NotebookLLM Magics loaded. Use %llm_config and %%llm.
   For ambient mode, try %llm_config_persistent to process all cells as LLM prompts.
❌ Still couldn't register NotebookLLMMagics


## 4. Verify the specific methods we need

In [4]:
# Find the magic instance and check for the required method
magics_instance = None

for magic_type, magics in ip.magics_manager.magics.items():
    for name, instance in magics.items():
        if isinstance(instance, NotebookLLMMagics):
            magics_instance = instance
            break
    if magics_instance:
        break

if magics_instance:
    if hasattr(magics_instance, "process_cell_as_prompt"):
        print("✅ process_cell_as_prompt method found")
    else:
        print("❌ process_cell_as_prompt method not found on NotebookLLMMagics instance")
        print(
            "Available methods:",
            [method for method in dir(magics_instance) if not method.startswith("_")],
        )
else:
    print("❌ No NotebookLLMMagics instance found")

❌ No NotebookLLMMagics instance found


## 5. Try enabling ambient mode

In [5]:
# Enable ambient mode
%llm_config_persistent

# Check if it's enabled
from cellmage.ambient_mode import is_ambient_mode_enabled

if is_ambient_mode_enabled():
    print("✅ Ambient mode is enabled")
else:
    print("❌ Ambient mode is NOT enabled")

--- NotebookLLM Status ---
Session ID: 990c4683-1fdf-43f3-b747-efb285df4600
None
Active Overrides: {'api_key': 'sk-L...mA', 'api_base': 'https://litellm.oracle.madpin.dev', 'model': 'gpt-4.1-nano'}
History Length: 0 messages
--------------------------
✅ Ambient mode ENABLED. All cells will now be processed as LLM prompts unless they start with % or !.
   Run %disable_llm_config_persistent to disable ambient mode.
✅ Ambient mode is enabled


## 6. Create a manual test for ambient mode

If the NotebookLLMMagics instance still can't be found by the transformer, let's create a way to manually execute the process_cell_as_prompt method.

In [6]:
# Create a manual helper function to process cells
def manual_process_as_prompt(prompt_text):
    """
    Manually process text as a prompt using the magics instance if available.
    This bypasses the input transformer mechanism.
    """
    ip = get_ipython()
    magics_instance = None

    # Find the registered instance
    for magic_type, magics in ip.magics_manager.magics.items():
        for instance in magics.values():
            if isinstance(instance, NotebookLLMMagics):
                magics_instance = instance
                break
        if magics_instance:
            break

    # If no registered instance, create a temporary one
    if not magics_instance:
        print("Creating temporary NotebookLLMMagics instance...")
        try:
            magics_instance = NotebookLLMMagics(ip)
        except Exception as e:
            print(f"❌ Error creating instance: {e}")
            return

    # Process the prompt
    if hasattr(magics_instance, "process_cell_as_prompt"):
        print("✅ Processing prompt...")
        magics_instance.process_cell_as_prompt(prompt_text)
    else:
        print("❌ process_cell_as_prompt method not found")

Your function `manual_process_as_prompt` is designed to locate an existing `NotebookLLMMagics` instance within the IPython environment and invoke its `process_cell_as_prompt` method to process a given prompt string.

Here are some suggestions to refine and ensure robustness:

1. **Import and Class Check:**  
   Make sure that `NotebookLLMMagics` is imported or defined in the scope where this function runs.

2. **Loop Logic:**  
   Currently, the nested loops break only upon finding the instance. To be more efficient, consider using a flag or a function to locate the instance.

3. **Error Handling:**  
   When creating a new instance, catching exceptions is good. Still, ensure `NotebookLLMMagics` can be instantiated without arguments or adapt accordingly.

4. **Documentation and Usage:**  
   Add clear docstrings and usage instructions.

Here's a refined version of your function with comments:

```python
def manual_process_as_prompt(prompt_text):
    """
    Manually process text as a prompt using the magics instance if available.
    This bypasses the input transformer mechanism.
    
    Parameters:
        prompt_text (str): The prompt string to process.
    """
    ip = get_ipython()
    magics_instance = None

    # Ensure NotebookLLMMagics is imported; replace with actual import if needed
    # from your_module import NotebookLLMMagics

    # Search for an existing instance in registered magics
    for magic_type, magics_dict in ip.magics_manager.magics.items():
        for name, instance in magics_dict.items():
            if isinstance(instance, NotebookLLMMagics):
                magics_instance = instance
                break
        if magics_instance:
            break

    # If no existing instance, create a temporary one
    if not magics_instance:
        print("Creating temporary NotebookLLMMagics instance...")
        try:
            magics_instance = NotebookLLMMagics(ip)
        except Exception as e:
            print(f"❌ Error creating instance: {e}")
            return

    # Process the prompt
    if hasattr(magics_instance, "process_cell_as_prompt"):
        print("✅ Processing prompt...")
        magics_instance.process_cell_as_prompt(prompt_text)
    else:
        print("❌ process_cell_as_prompt method not found")
```

**Notes:**

- Replace placeholder comments with actual imports as needed.
- Confirm that `NotebookLLMMagics` can be instantiated with `ip` as an argument.
- You might consider adding additional error handling or logging depending on your environment.

Let me know if you need further enhancements or integration suggestions!

In [7]:
# Test the manual processing function
manual_process_as_prompt("Explain how ambient mode works in CellMage.")

To test the `manual_process_as_prompt` function with the prompt `"Explain how ambient mode works in CellMage."`, follow these steps:

1. **Ensure that all dependencies are imported and available in your environment.**  
   - Confirm that `get_ipython()` is available (it should be in an IPython or Jupyter notebook).  
   - Confirm that `NotebookLLMMagics` is imported or defined in your environment.

2. **Run the test command.**  
   Simply execute:

```python
manual_process_as_prompt("Explain how ambient mode works in CellMage.")
```

3. **Observe the output.**  
   - If an existing `NotebookLLMMagics` instance is found, it will process the prompt. You should see the message `"✅ Processing prompt..."` and then the processing output.  
   - If not, it will create a temporary instance and then proceed.  
   - Any errors during creation or processing will be printed accordingly.

---

### Example of how it should look in your environment:

```python
# Run the function with the test prompt
manual_process_as_prompt("Explain how ambient mode works in CellMage.")
```

### Expected output (example):

```plaintext
✅ Processing prompt...
# [Output from process_cell_as_prompt method]
```

or, if an error occurs:

```plaintext
❌ Error creating instance: <error message>
```

---

### Additional tips:
- Make sure that your environment has `NotebookLLMMagics` properly defined and capable of `process_cell_as_prompt`.
- If no output appears, verify that `process_cell_as_prompt` produces visible output or side-effects.
- You can add print statements inside `process_cell_as_prompt` to debug or confirm it's being called.

Let me know if you encounter any issues during testing!

## 7. Create a stealthy hook for ambient mode

If we still have issues with the input transformer, let's create an alternative approach using IPython's post_run_cell hook.

In [8]:
# Alternative approach using IPython hooks
ip = get_ipython()

# Remove any existing hook first
if hasattr(ip, "_stealthy_ambient_mode_hook"):
    ip.events.unregister("post_run_cell", ip._stealthy_ambient_mode_hook)


# Define our hook function
def ambient_mode_hook(result):
    # Skip if no result or result has error
    if not result or result.error_in_exec:
        return

    # Get the cell content
    cell = ip.user_ns.get("_i" + str(ip.execution_count))
    if not cell or not cell.strip():
        return

    # Skip if it starts with % or ! or %%
    if cell.strip().startswith(("%", "!", "%%")):
        return

    # Skip if it's a Python statement (has =, def, class, import, etc.)
    python_keywords = ["def ", "class ", "import ", " = ", "return ", "if ", "for ", "while "]
    if any(keyword in cell for keyword in python_keywords):
        return

    # Process as prompt if it looks like natural language
    manual_process_as_prompt(cell)


# Register our hook
ip._stealthy_ambient_mode_hook = ambient_mode_hook
ip.events.register("post_run_cell", ambient_mode_hook)

print("✅ Alternative ambient mode hook registered")

Your approach sets up a custom IPython hook that automatically processes cells as prompts after each cell run, based on certain heuristics. Here's a brief overview and some considerations:

### What this code does:

- **Unregisters previous hook** if it exists, preventing duplicates.
- **Defines** `ambient_mode_hook`, which:
  - Checks whether the cell is suitable for processing (not a magic command or shell command, not a Python statement).
  - Looks for natural language cells to process.
  - Calls your `manual_process_as_prompt` function on the cell content.
- **Registers** the hook to run after each cell execution.

### Usage instructions:

1. **Define the hook code** in your environment (e.g., a cell in a Jupyter notebook).
2. **Run the code to register the hook.**
3. **Execute cells normally.**  
   When a cell's content looks like natural language (e.g., a question or explanation), it will automatically trigger your processing.

---

### Important notes:

- This setup **automatically processes cells** after execution, which might be resource-intensive if many cells run frequently.
- The heuristics used to filter cells (like checking for starting symbols or Python keywords) can be tuned to better match your use case.
- Make sure `manual_process_as_prompt` is correctly defined in scope.

---

### Suggested improvements:

- You could add more sophisticated detection (e.g., NLP-based detection of natural language queries).
- To avoid processing important code cells accidentally, refine your heuristics.
- To prevent repeated processing or infinite loops, you might want to add flags or tags.

---

### Final step:

Just run this setup code in your environment:

```python
import sys

ip = get_ipython()

# Remove any existing hook first
if hasattr(ip, "_stealthy_ambient_mode_hook"):
    try:
        ip.events.unregister("post_run_cell", ip._stealthy_ambient_mode_hook)
    except Exception:
        pass

# Define the hook function
def ambient_mode_hook(result):
    # Skip if no result or error
    if not result or result.error_in_exec:
        return

    # Get the last executed cell content
    cell = ip.user_ns.get("_i" + str(ip.execution_count))
    if not cell or not cell.strip():
        return

    stripped_cell = cell.strip()

    # Skip magic and shell commands
    if stripped_cell.startswith(("%", "!", "%%")):
        return

    # Skip typical Python statements
    python_keywords = ["def ", "class ", "import ", " = ", "return ", "if ", "for ", "while "]
    if any(keyword in stripped_cell for keyword in python_keywords):
        return

    # Process as prompt if likely natural language
    manual_process_as_prompt(cell)

# Register the hook
ip._stealthy_ambient_mode_hook = ambient_mode_hook
ip.events.register("post_run_cell", ambient_mode_hook)

print("✅ Alternative ambient mode hook registered")
```

---

### Test the setup:

- Run some cells containing natural language questions or explanations.
- Observe whether `manual_process_as_prompt` is invoked automatically.
- Adjust heuristics as needed for your use case.

---

If you'd like, I can help you refine the heuristics further or add features!

## 8. Test our custom ambient mode

Now cells that look like natural language and don't contain Python code should be processed as prompts automatically.

In [9]:
# This should be processed as a prompt
What is CellMage and how does it help with notebook workflows?

This message is a natural language question, so in your setup, the hook should automatically detect it as a candidate for processing and call `manual_process_as_prompt` with this content.

**Expected behavior:**

- The hook runs after cell execution.
- It detects that the cell looks like a natural language query.
- It invokes `manual_process_as_prompt` with the cell content.
- Your `process_cell_as_prompt` method in `NotebookLLMMagics` processes this prompt accordingly.

---

### What to do next:

1. **Run the setup code** provided previously to register the hook.
2. **Input or run the cell** with the question:

```plaintext
What is CellMage and how does it help with notebook workflows?
```

3. **Observe the output:**  
   Your magic's `process_cell_as_prompt` method should handle generating a response or performing the desired processing.

---

### Summary:

Your environment is configured to act as an assistant, passing natural language queries to your language model handler automatically. You can continue asking questions or describing tasks, and the system should respond accordingly.

In [10]:
# This should NOT be processed as a prompt
x = 10
y = 20
print("This is normal Python code:", x + y)

In your setup, the hook is designed to **skip processing** cells that are recognized as Python code or start with magic or shell commands. 

**Based on your code:**

- The cell content is:

```python
x = 10

y = 20

print("This is normal Python code:", x + y)
```

- The hook checks for Python keywords like `"def "`, `"class "`, `"import "`, `" = "`, `"return "`, `"if "`, `"for "`, `"while "`.

- Since the cell contains assignments and a print statement, it matches these heuristics (e.g., `" = "` appears multiple times).

**Therefore:**

- The hook **will not** process this cell as a prompt.
- It will **skip invoking** `manual_process_as_prompt`.

---

### To test or verify:

- Run this cell.
- Observe that no processing is triggered.
- The code executes normally, printing:

```plaintext
This is normal Python code: 30
```

---

### Summary:

Your current filtering heuristics effectively prevent code cells from being processed as prompts. This setup aims to process only natural language or potential questions, avoiding interference with regular code execution.

## 9. Disable ambient mode

In [11]:
# Disable both approaches to ambient mode

# 1. Standard transformer approach
%disable_llm_config_persistent

# 2. Our custom hook approach
if hasattr(ip, '_stealthy_ambient_mode_hook'):
    ip.events.unregister('post_run_cell', ip._stealthy_ambient_mode_hook)
    print("✅ Removed custom ambient mode hook")

# Check if ambient mode is disabled
if not is_ambient_mode_enabled():
    print("✅ Official ambient mode is disabled")
else:
    print("❌ Official ambient mode is still enabled")

❌ Ambient mode DISABLED. Regular cells will now be executed normally.
✅ Official ambient mode is disabled


## 8. Troubleshooting

If you're experiencing the "Could not find registered NotebookLLMMagics instance" error, try the following solutions:

1. Ensure the extension is loaded by running `%load_ext cellmage.integrations.ipython_magic` first
2. Try restarting the kernel and loading the extension again
3. Check if there are any errors during the extension loading process
4. Verify that the package is installed correctly with `!pip show cellmage`

A common issue with the ambient mode is that it depends on finding the registered NotebookLLMMagics instance in the IPython environment. The error occurs when the code that's injected by the input transformer can't find this instance.